In [1]:
import random

import numpy as np
import torch
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from torch import nn
from tqdm import tqdm
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader

In [2]:
data = np.load(r'D:\heat_wave\pacific\last\wind_93_19_pacific_area.npz')
print(data.files)  # ['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']

eastward_wind = data['eastward_wind'][:, :40, :200]  # (9861, 41, 201)  #
print(eastward_wind.shape)  # (9861, 41, 201)
northward_wind = data['northward_wind'][:, :40, :200]  # (9861, 41, 201)
wind_vector_divergence = data['wind_vector_divergence'][:, :40, :200]  # (9861, 41, 201)
wind_stress = data['wind_stress'][:, :40, :200]  # (9861, 41, 201)   # 可惜这个是 0
surface_downward_eastward_stress = data['surface_downward_eastward_stress'][:, :40, :200]  # (9861, 41, 201)
surface_downward_northward_stress = data['surface_downward_northward_stress'][:, :40, :200]  # (9861, 41, 201)
wind_stress_curl = data['wind_stress_curl'][:, :40, :200]  # (9861, 41, 201)
wind_stress_divergence = data['wind_stress_divergence'][:, :40, :200]  # (9861, 41, 201)
wind_speed_rms = data['wind_speed_rms'][:, :40, :200]  # (9861, 41, 201)
eastward_wind_rms = data['eastward_wind_rms'][:, :40, :200]  # (9861, 41, 201)
northward_wind_rms = data['northward_wind_rms'][:, :40, :200]  # (9861, 41, 201)
sampling_length = data['sampling_length'][:, :40, :200]  # (9861, 41, 201)
surface_type = data['surface_type'][:, :40, :200]  # (9861, 41, 201)

['time', 'lat', 'lon', 'eastward_wind', 'northward_wind', 'wind_vector_divergence', 'wind_stress', 'surface_downward_eastward_stress', 'surface_downward_northward_stress', 'wind_stress_curl', 'wind_stress_divergence', 'wind_speed_rms', 'eastward_wind_rms', 'northward_wind_rms', 'sampling_length', 'surface_type']
(9848, 40, 200)


In [3]:
# 首先 我们先画 DJF季节的贡献图   DJF： 12  1  2 月  即  12/01 - 02/31  当然，会做一个区域平均
# 同时，我们也会对每一年的DJF做一个平均  ---》1993 - 2019年

In [4]:
#  计算索引
import pandas as pd
import datetime

#创建时间范围
start_date = datetime.datetime(1993,1,1)
end_date = datetime.datetime(2019,12,31)
dates = pd.date_range(start = start_date, end = end_date, freq = 'D')

index = dates.get_loc(datetime.datetime(2019,8,31)) 
index

9738

In [5]:
# DJF   93
time_93_1 = 0
time_93_2 = 59
data_1 = time_93_1
data_2 = time_93_2

eastward_wind_DJF_93 = eastward_wind[data_1 : data_2,:,:]
northward_wind_DJF_93 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_DJF_93 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_DJF_93 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_93 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_93 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_DJF_93 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_DJF_93 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_DJF_93 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_DJF_93 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_93 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_DJF_93 = sampling_length[data_1 : data_2,:,:]
surface_type_DJF_93 = surface_type[data_1 : data_2,:,:]

In [6]:
# DJF   94
time_94_1 = 334
time_94_2 = 424
data_1 = time_94_1
data_2 = time_94_2

eastward_wind_DJF_94 = eastward_wind[data_1 : data_2,:,:]
northward_wind_DJF_94 = northward_wind[data_1 : data_2,:,:]
wind_vector_divergence_DJF_94 = wind_vector_divergence[data_1 : data_2,:,:]
wind_stress_DJF_94 = wind_stress[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_94 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_94 = surface_downward_northward_stress[data_1 : data_2,:,:]
wind_stress_curl_DJF_94 = wind_stress_curl[data_1 : data_2,:,:]
wind_stress_divergence_DJF_94 = wind_stress_divergence[data_1 : data_2,:,:]
wind_speed_rms_DJF_94 = wind_speed_rms[data_1 : data_2,:,:]
eastward_wind_rms_DJF_94 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_94 = northward_wind_rms[data_1 : data_2,:,:]
sampling_length_DJF_94 = sampling_length[data_1 : data_2,:,:]
surface_type_DJF_94 = surface_type[data_1 : data_2,:,:]

In [7]:
# DJF   95
time_95_1 = 699
time_95_2 = 789
data_1 = time_95_1
data_2 = time_95_2

wind_stress_DJF_95 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_95 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_95 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_95 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_95 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_95 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_95 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_95 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_95 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_95 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_95 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_95 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_95 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [8]:
# DJF   96
time_96_1 = 1064
time_96_2 = 1155
data_1 = time_96_1
data_2 = time_96_2

wind_stress_DJF_96 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_96 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_96 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_96 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_96 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_96 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_96 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_96 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_96 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_96 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_96 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_96 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_96 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [9]:
# DJF   97
time_97_1 = 1430
time_97_2 = 1520
data_1 = time_97_1
data_2 = time_97_2

wind_stress_DJF_97 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_97 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_97 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_97 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_97 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_97 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_97 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_97 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_97 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_97 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_97 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_97 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_97 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [10]:
# DJF   98
time_98_1 = 1795
time_98_2 = 1885
data_1 = time_98_1
data_2 = time_98_2

wind_stress_DJF_98 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_98 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_98 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_98 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_98 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_98 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_98 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_98 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_98 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_98 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_98 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_98 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_98 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [11]:
# DJF   99
time_99_1 = 2160
time_99_2 = 2250
data_1 = time_99_1
data_2 = time_99_2

wind_stress_DJF_99 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_99 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_99 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_99 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_99 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_99 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_99 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_99 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_99 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_99 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_99 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_99 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_99 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [12]:
# DJF   00
time_00_1 = 2525
time_00_2 = 2616
data_1 = time_00_1
data_2 = time_00_2

wind_stress_DJF_00 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_00 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_00 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_00 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_00 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_00 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_00 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_00 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_00 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_00 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_00 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_00 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_00 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [13]:
# DJF   01
time_01_1 = 2891
time_01_2 = 2981
data_1 = time_01_1
data_2 = time_01_2

wind_stress_DJF_01 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_01 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_01 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_01 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_01 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_01 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_01 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_01 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_01 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_01 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_01 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_01 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_01 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [14]:
# DJF   02
time_02_1 = 3256
time_02_2 = 3346
data_1 = time_02_1
data_2 = time_02_2

wind_stress_DJF_02 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_02 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_02 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_02 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_02 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_02 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_02 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_02 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_02 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_02 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_02 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_02 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_02 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [15]:
# DJF   03
time_03_1 = 3621
time_03_2 = 3711
data_1 = time_03_1
data_2 = time_03_2

wind_stress_DJF_03 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_03 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_03 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_03 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_03 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_03 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_03 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_03 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_03 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_03 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_03 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_03 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_03 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [16]:
# DJF   04
time_04_1 = 3986
time_04_2 = 4077
data_1 = time_04_1
data_2 = time_04_2

wind_stress_DJF_04 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_04 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_04 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_04 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_04 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_04 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_04 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_04 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_04 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_04 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_04 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_04 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_04 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [17]:
# DJF   05
time_05_1 = 4352
time_05_2 = 4442
data_1 = time_05_1
data_2 = time_05_2

wind_stress_DJF_05 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_05 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_05 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_05 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_05 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_05 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_05 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_05 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_05 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_05 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_05 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_05 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_05 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [18]:
# DJF   06
time_06_1 = 4717
time_06_2 = 4807
data_1 = time_06_1
data_2 = time_06_2

wind_stress_DJF_06 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_06 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_06 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_06 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_06 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_06 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_06 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_06 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_06 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_06 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_06 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_06 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_06 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [19]:
# DJF   07
time_07_1 = 5082
time_07_2 = 5172
data_1 = time_07_1
data_2 = time_07_2

wind_stress_DJF_07 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_07 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_07 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_07 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_07 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_07 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_07 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_07 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_07 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_07 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_07 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_07 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_07 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [20]:
# DJF   08
time_08_1 = 5447
time_08_2 = 5538
data_1 = time_08_1
data_2 = time_08_2

wind_stress_DJF_08 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_08 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_08 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_08 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_08 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_08 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_08 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_08 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_08 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_08 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_08 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_08 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_08 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [21]:
# DJF   09
time_09_1 = 5813
time_09_2 = 5903
data_1 = time_09_1
data_2 = time_09_2

wind_stress_DJF_09 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_09 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_09 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_09 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_09 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_09 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_09 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_09 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_09 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_09 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_09 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_09 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_09 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [22]:
# DJF   10
time_10_1 = 6178
time_10_2 = 6268
data_1 = time_10_1
data_2 = time_10_2

wind_stress_DJF_10 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_10 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_10 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_10 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_10 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_10 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_10 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_10 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_10 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_10 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_10 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_10 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_10 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [23]:
# DJF   11
time_11_1 = 6543
time_11_2 = 6633
data_1 = time_11_1
data_2 = time_11_2

wind_stress_DJF_11 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_11 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_11 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_11 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_11 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_11 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_11 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_11 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_11 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_11 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_11 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_11 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_11 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [24]:
# DJF   12
time_12_1 = 6908
time_12_2 = 6999
data_1 = time_12_1
data_2 = time_12_2

wind_stress_DJF_12 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_12 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_12 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_12 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_12 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_12 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_12 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_12 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_12 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_12 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_12 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_12 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_12 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [25]:
# DJF   13
time_13_1 = 7274
time_13_2 = 7364
data_1 = time_13_1
data_2 = time_13_2

wind_stress_DJF_13 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_13 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_13 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_13 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_13 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_13 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_13 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_13 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_13 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_13 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_13 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_13 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_13 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [26]:
# DJF   14
time_14_1 = 7639
time_14_2 = 7729
data_1 = time_14_1
data_2 = time_14_2

wind_stress_DJF_14 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_14 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_14 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_14 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_14 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_14 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_14 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_14 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_14 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_14 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_14 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_14 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_14 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [27]:
# DJF   15
time_15_1 = 8004
time_15_2 = 8094
data_1 = time_15_1
data_2 = time_15_2

wind_stress_DJF_15 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_15 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_15 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_15 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_15 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_15 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_15 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_15 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_15 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_15 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_15 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_15 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_15 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [28]:
# DJF   16
time_16_1 = 8369
time_16_2 = 8460
data_1 = time_16_1
data_2 = time_16_2

wind_stress_DJF_16 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_16 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_16 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_16 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_16 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_16 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_16 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_16 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_16 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_16 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_16 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_16 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_16 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [29]:
# DJF   17
time_17_1 = 8735
time_17_2 = 8825
data_1 = time_17_1
data_2 = time_17_2

wind_stress_DJF_17 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_17 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_17 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_17 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_17 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_17 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_17 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_17 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_17 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_17 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_17 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_17 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_17 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [30]:
# DJF   18
time_18_1 = 9100
time_18_2 = 9190
data_1 = time_18_1
data_2 = time_18_2

wind_stress_DJF_18 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_18 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_18 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_18 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_18 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_18 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_18 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_18 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_18 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_18 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_18 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_18 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_18 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [31]:
# DJF   19
time_19_1 = 9452
time_19_2 = 9542
data_1 = time_19_1
data_2 = time_19_2

wind_stress_DJF_19 = wind_stress[data_1 : data_2,:,:]
surface_type_DJF_19 = surface_type[data_1 : data_2,:,:]
eastward_wind_DJF_19 = eastward_wind[data_1 : data_2,:,:]
wind_stress_curl_DJF_19 = wind_stress_curl[data_1 : data_2,:,:]
northward_wind_DJF_19 = northward_wind[data_1 : data_2,:,:]
wind_speed_rms_DJF_19 = wind_speed_rms[data_1 : data_2,:,:]
sampling_length_DJF_19 = sampling_length[data_1 : data_2,:,:]
eastward_wind_rms_DJF_19 = eastward_wind_rms[data_1 : data_2,:,:]
northward_wind_rms_DJF_19 = northward_wind_rms[data_1 : data_2,:,:]
wind_stress_divergence_DJF_19 = wind_stress_divergence[data_1 : data_2,:,:]
wind_vector_divergence_DJF_19 = wind_vector_divergence[data_1 : data_2,:,:]
surface_downward_eastward_stress_DJF_19 = surface_downward_eastward_stress[data_1 : data_2,:,:]
surface_downward_northward_stress_DJF_19 = surface_downward_northward_stress[data_1 : data_2,:,:]

In [32]:
wind_stress_DJF = np.concatenate((wind_stress_DJF_93, wind_stress_DJF_94, wind_stress_DJF_95,
                                 wind_stress_DJF_96, wind_stress_DJF_97, wind_stress_DJF_98, wind_stress_DJF_99,
                                 wind_stress_DJF_00, wind_stress_DJF_01, wind_stress_DJF_02, wind_stress_DJF_03,
                                 wind_stress_DJF_04, wind_stress_DJF_05, wind_stress_DJF_06, wind_stress_DJF_07,
                                 wind_stress_DJF_08, wind_stress_DJF_09, wind_stress_DJF_10, wind_stress_DJF_11,
                                 wind_stress_DJF_12, wind_stress_DJF_13, wind_stress_DJF_14, wind_stress_DJF_15,
                                 wind_stress_DJF_16, wind_stress_DJF_17, wind_stress_DJF_18, wind_stress_DJF_19), axis = 0)

In [33]:
surface_type_DJF = np.concatenate((surface_type_DJF_93, surface_type_DJF_94, surface_type_DJF_95,
                                  surface_type_DJF_96, surface_type_DJF_97, surface_type_DJF_98, surface_type_DJF_99,
                                  surface_type_DJF_00, surface_type_DJF_01, surface_type_DJF_02, surface_type_DJF_03,
                                  surface_type_DJF_04, surface_type_DJF_05, surface_type_DJF_06, surface_type_DJF_07,
                                  surface_type_DJF_08, surface_type_DJF_09, surface_type_DJF_10, surface_type_DJF_11,
                                  surface_type_DJF_12, surface_type_DJF_13, surface_type_DJF_14, surface_type_DJF_15,
                                  surface_type_DJF_16, surface_type_DJF_17, surface_type_DJF_18, surface_type_DJF_19), axis = 0)

In [34]:
eastward_wind_DJF = np.concatenate((eastward_wind_DJF_93, eastward_wind_DJF_94, eastward_wind_DJF_95,
                                   eastward_wind_DJF_96, eastward_wind_DJF_97, eastward_wind_DJF_98,
                                   eastward_wind_DJF_99, eastward_wind_DJF_00, eastward_wind_DJF_01,
                                   eastward_wind_DJF_02, eastward_wind_DJF_03, eastward_wind_DJF_04,
                                   eastward_wind_DJF_05, eastward_wind_DJF_06, eastward_wind_DJF_07,
                                   eastward_wind_DJF_08, eastward_wind_DJF_09, eastward_wind_DJF_10,
                                   eastward_wind_DJF_11, eastward_wind_DJF_12, eastward_wind_DJF_13,
                                   eastward_wind_DJF_14, eastward_wind_DJF_15, eastward_wind_DJF_16,
                                   eastward_wind_DJF_17, eastward_wind_DJF_18, eastward_wind_DJF_19), axis = 0)

In [35]:
wind_stress_curl_DJF = np.concatenate((wind_stress_curl_DJF_93, wind_stress_curl_DJF_94, wind_stress_curl_DJF_95,
                                      wind_stress_curl_DJF_96, wind_stress_curl_DJF_97, wind_stress_curl_DJF_98, 
                                      wind_stress_curl_DJF_99, wind_stress_curl_DJF_00, wind_stress_curl_DJF_01,
                                      wind_stress_curl_DJF_02, wind_stress_curl_DJF_03, wind_stress_curl_DJF_04, 
                                      wind_stress_curl_DJF_05, wind_stress_curl_DJF_06, wind_stress_curl_DJF_07,
                                      wind_stress_curl_DJF_08, wind_stress_curl_DJF_09, wind_stress_curl_DJF_10,
                                      wind_stress_curl_DJF_11, wind_stress_curl_DJF_12, wind_stress_curl_DJF_13,
                                      wind_stress_curl_DJF_14, wind_stress_curl_DJF_15, wind_stress_curl_DJF_16,
                                      wind_stress_curl_DJF_17, wind_stress_curl_DJF_18, wind_stress_curl_DJF_19), axis = 0)

In [36]:
northward_wind_DJF = np.concatenate((northward_wind_DJF_93, northward_wind_DJF_94, northward_wind_DJF_95,
                                    northward_wind_DJF_96, northward_wind_DJF_97, northward_wind_DJF_98,
                                    northward_wind_DJF_99, northward_wind_DJF_00, northward_wind_DJF_01,
                                    northward_wind_DJF_02, northward_wind_DJF_03, northward_wind_DJF_04, 
                                    northward_wind_DJF_05, northward_wind_DJF_06, northward_wind_DJF_07,
                                    northward_wind_DJF_08, northward_wind_DJF_09, northward_wind_DJF_10,
                                    northward_wind_DJF_11, northward_wind_DJF_12, northward_wind_DJF_13,
                                    northward_wind_DJF_14, northward_wind_DJF_15, northward_wind_DJF_16,
                                    northward_wind_DJF_17, northward_wind_DJF_18, northward_wind_DJF_19), axis = 0)

In [37]:
wind_speed_rms_DJF = np.concatenate((wind_speed_rms_DJF_93, wind_speed_rms_DJF_94, wind_speed_rms_DJF_95,
                                    wind_speed_rms_DJF_96, wind_speed_rms_DJF_97, wind_speed_rms_DJF_98,
                                    wind_speed_rms_DJF_99, wind_speed_rms_DJF_00, wind_speed_rms_DJF_01,
                                    wind_speed_rms_DJF_02, wind_speed_rms_DJF_03, wind_speed_rms_DJF_04,
                                    wind_speed_rms_DJF_05, wind_speed_rms_DJF_06, wind_speed_rms_DJF_07,
                                    wind_speed_rms_DJF_08, wind_speed_rms_DJF_09, wind_speed_rms_DJF_10,
                                    wind_speed_rms_DJF_11, wind_speed_rms_DJF_12, wind_speed_rms_DJF_13,
                                    wind_speed_rms_DJF_14, wind_speed_rms_DJF_15, wind_speed_rms_DJF_16,
                                    wind_speed_rms_DJF_17, wind_speed_rms_DJF_18, wind_speed_rms_DJF_19), axis = 0)

In [38]:
sampling_length_DJF = np.concatenate((sampling_length_DJF_93, sampling_length_DJF_94, sampling_length_DJF_95,
                                     sampling_length_DJF_96, sampling_length_DJF_97, sampling_length_DJF_98,
                                     sampling_length_DJF_99, sampling_length_DJF_00, sampling_length_DJF_01,
                                     sampling_length_DJF_02, sampling_length_DJF_03, sampling_length_DJF_04,
                                     sampling_length_DJF_05, sampling_length_DJF_06, sampling_length_DJF_07,
                                     sampling_length_DJF_08, sampling_length_DJF_09, sampling_length_DJF_10,
                                     sampling_length_DJF_11, sampling_length_DJF_12, sampling_length_DJF_13,
                                     sampling_length_DJF_14, sampling_length_DJF_15, sampling_length_DJF_16,
                                     sampling_length_DJF_17, sampling_length_DJF_18, sampling_length_DJF_19), axis = 0)

In [39]:
eastward_wind_rms_DJF = np.concatenate((eastward_wind_rms_DJF_93, eastward_wind_rms_DJF_94, eastward_wind_rms_DJF_95,
                                       eastward_wind_rms_DJF_96, eastward_wind_rms_DJF_97, eastward_wind_rms_DJF_98,
                                       eastward_wind_rms_DJF_99, eastward_wind_rms_DJF_00, eastward_wind_rms_DJF_01,
                                       eastward_wind_rms_DJF_02, eastward_wind_rms_DJF_03, eastward_wind_rms_DJF_04,
                                       eastward_wind_rms_DJF_05, eastward_wind_rms_DJF_06, eastward_wind_rms_DJF_07,
                                       eastward_wind_rms_DJF_08, eastward_wind_rms_DJF_09, eastward_wind_rms_DJF_10,
                                       eastward_wind_rms_DJF_11, eastward_wind_rms_DJF_12, eastward_wind_rms_DJF_13,
                                       eastward_wind_rms_DJF_14, eastward_wind_rms_DJF_15, eastward_wind_rms_DJF_16,
                                       eastward_wind_rms_DJF_17, eastward_wind_rms_DJF_18, eastward_wind_rms_DJF_19), axis = 0)

In [40]:
northward_wind_rms_DJF = np.concatenate((northward_wind_rms_DJF_93, northward_wind_rms_DJF_94,
                                        northward_wind_rms_DJF_95, northward_wind_rms_DJF_96,northward_wind_rms_DJF_97,
                                        northward_wind_rms_DJF_98, northward_wind_rms_DJF_99, northward_wind_rms_DJF_00,
                                        northward_wind_rms_DJF_01, northward_wind_rms_DJF_02, northward_wind_rms_DJF_03,
                                        northward_wind_rms_DJF_04, northward_wind_rms_DJF_05, northward_wind_rms_DJF_06,
                                        northward_wind_rms_DJF_07, northward_wind_rms_DJF_08, northward_wind_rms_DJF_09,
                                        northward_wind_rms_DJF_10, northward_wind_rms_DJF_11, northward_wind_rms_DJF_12,
                                        northward_wind_rms_DJF_13, northward_wind_rms_DJF_14, northward_wind_rms_DJF_15,
                                        northward_wind_rms_DJF_16, northward_wind_rms_DJF_17, northward_wind_rms_DJF_18,
                                        northward_wind_rms_DJF_19), axis = 0)

In [41]:
wind_stress_divergence_DJF = np.concatenate((wind_stress_divergence_DJF_93, wind_stress_divergence_DJF_94,
                                            wind_stress_divergence_DJF_95, wind_stress_divergence_DJF_96, 
                                            wind_stress_divergence_DJF_97, wind_stress_divergence_DJF_98,
                                            wind_stress_divergence_DJF_99, wind_stress_divergence_DJF_00,
                                            wind_stress_divergence_DJF_01, wind_stress_divergence_DJF_02,
                                            wind_stress_divergence_DJF_03, wind_stress_divergence_DJF_04,
                                            wind_stress_divergence_DJF_05, wind_stress_divergence_DJF_06, 
                                            wind_stress_divergence_DJF_07, wind_stress_divergence_DJF_08,
                                            wind_stress_divergence_DJF_09, wind_stress_divergence_DJF_10,
                                            wind_stress_divergence_DJF_11, wind_stress_divergence_DJF_12,
                                            wind_stress_divergence_DJF_13, wind_stress_divergence_DJF_14,
                                            wind_stress_divergence_DJF_15, wind_stress_divergence_DJF_16,
                                            wind_stress_divergence_DJF_17, wind_stress_divergence_DJF_18, wind_stress_divergence_DJF_19), axis = 0)

In [42]:
wind_vector_divergence_DJF = np.concatenate((wind_vector_divergence_DJF_93, wind_vector_divergence_DJF_94,
                                            wind_vector_divergence_DJF_95, wind_vector_divergence_DJF_96,
                                            wind_vector_divergence_DJF_97, wind_vector_divergence_DJF_98,
                                            wind_vector_divergence_DJF_99, wind_vector_divergence_DJF_00,
                                            wind_vector_divergence_DJF_01, wind_vector_divergence_DJF_02,
                                            wind_vector_divergence_DJF_03, wind_vector_divergence_DJF_04,
                                            wind_vector_divergence_DJF_05, wind_vector_divergence_DJF_06,
                                            wind_vector_divergence_DJF_07, wind_vector_divergence_DJF_08,
                                            wind_vector_divergence_DJF_09, wind_vector_divergence_DJF_10,
                                            wind_vector_divergence_DJF_11, wind_vector_divergence_DJF_12,
                                            wind_vector_divergence_DJF_13, wind_vector_divergence_DJF_14,
                                            wind_vector_divergence_DJF_15, wind_vector_divergence_DJF_16,
                                            wind_vector_divergence_DJF_17, wind_vector_divergence_DJF_18, wind_vector_divergence_DJF_19), axis = 0)

In [43]:
surface_downward_eastward_stress_DJF = np.concatenate((surface_downward_eastward_stress_DJF_93,
                                                      surface_downward_eastward_stress_DJF_94, surface_downward_eastward_stress_DJF_95,
                                                      surface_downward_eastward_stress_DJF_96, surface_downward_eastward_stress_DJF_97,
                                                      surface_downward_eastward_stress_DJF_98, surface_downward_eastward_stress_DJF_99,
                                                      surface_downward_eastward_stress_DJF_00, surface_downward_eastward_stress_DJF_01,
                                                      surface_downward_eastward_stress_DJF_02, surface_downward_eastward_stress_DJF_03,
                                                      surface_downward_eastward_stress_DJF_04, surface_downward_eastward_stress_DJF_05,
                                                      surface_downward_eastward_stress_DJF_06, surface_downward_eastward_stress_DJF_07,
                                                      surface_downward_eastward_stress_DJF_08, surface_downward_eastward_stress_DJF_09,
                                                      surface_downward_eastward_stress_DJF_10, surface_downward_eastward_stress_DJF_11,
                                                      surface_downward_eastward_stress_DJF_12, surface_downward_eastward_stress_DJF_13,
                                                      surface_downward_eastward_stress_DJF_14, surface_downward_eastward_stress_DJF_15,
                                                      surface_downward_eastward_stress_DJF_16, surface_downward_eastward_stress_DJF_17,
                                                      surface_downward_eastward_stress_DJF_18, surface_downward_eastward_stress_DJF_19), axis = 0)

In [44]:
surface_downward_northward_stress_DJF = np.concatenate((surface_downward_northward_stress_DJF_93,
                                                       surface_downward_northward_stress_DJF_94, surface_downward_northward_stress_DJF_95,
                                                       surface_downward_northward_stress_DJF_96, surface_downward_northward_stress_DJF_97,
                                                       surface_downward_northward_stress_DJF_98, surface_downward_northward_stress_DJF_99,
                                                       surface_downward_northward_stress_DJF_00, surface_downward_northward_stress_DJF_01,
                                                       surface_downward_northward_stress_DJF_02, surface_downward_northward_stress_DJF_03,
                                                       surface_downward_northward_stress_DJF_04, surface_downward_northward_stress_DJF_05,
                                                       surface_downward_northward_stress_DJF_06, surface_downward_northward_stress_DJF_07,
                                                       surface_downward_northward_stress_DJF_08, surface_downward_northward_stress_DJF_09,
                                                       surface_downward_northward_stress_DJF_10, surface_downward_northward_stress_DJF_11,
                                                       surface_downward_northward_stress_DJF_12, surface_downward_northward_stress_DJF_13,
                                                       surface_downward_northward_stress_DJF_14, surface_downward_northward_stress_DJF_15,
                                                       surface_downward_northward_stress_DJF_16, surface_downward_northward_stress_DJF_17,
                                                       surface_downward_northward_stress_DJF_18, surface_downward_northward_stress_DJF_19), axis = 0)

In [45]:
np.savez(r'D:\heat_wave\pacific\last\wind\wind_93_19_pacific_area_DJF.npz',
        wind_stress_DJF = wind_stress_DJF, surface_type_DJF = surface_type_DJF, eastward_wind_DJF = eastward_wind_DJF,
        wind_stress_curl_DJF = wind_stress_curl_DJF, northward_wind_DJF = northward_wind_DJF, 
        wind_speed_rms_DJF = wind_speed_rms_DJF, sampling_length_DJF = sampling_length_DJF,
        eastward_wind_rms_DJF = eastward_wind_rms_DJF, northward_wind_rms_DJF = northward_wind_rms_DJF,
        surface_downward_eastward_stress_DJF = surface_downward_eastward_stress_DJF,
        surface_downward_northward_stress_DJF = surface_downward_northward_stress_DJF)